<div>
    <h1>Large Language Models Projects</a></h1>
    <h3>Apply and Implement Strategies for Large Language Models</h3>
    <h2>3.2-Create a Moderation System using LangChain.</h2>
    <h3>LLAMA 2 version</h3>
  </div>

by [Pere Martra](https://www.linkedin.com/in/pere-martra/)

________
Model: Llama-2-7b-chat-hf

Colab Environment: GPU-T4 High-RAM. (You can use a more advanced and powerful GPU).

Keys:
* Lagchain
* Llama
* Moderator.

# How To Create a Moderation System Using LangChain & Hugging Face.

We are going to create a moderation system based on two models.

The first model reads user comments and generates responses.

The second language model then analyzes the generated response, identifying any negativity and modifying the response if necessary.

This process aims to prevent negative or inappropriate user input from triggering a similarly negative or off-tone response from the comment system.

In [2]:
#Install de LangChain and openai libraries.
!pip install -q langchain==0.1.4
!pip install -q transformers==4.37.1
!pip install -q accelerate==0.26.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.6/803.6 kB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 68.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.2/241.2 kB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 4.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
accelerate 1.2.1 requires huggingface-hub>=0.21.0, but you have huggingface-hub 0.20.2 which is incompatible.
langchain-text-splitters 0.3.3 requires langchain-core<0.4.0,>=0.3.25, but you have langchain-core 0.1.23 which is incompatible.
peft 0.14.0 requires huggingface-hub>=0.25.0, but you have huggingface-hub 0.20.2 which is incompatible.
transfor

In [1]:
!pip install huggingface_hub==0.20.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 330.3/330.3 kB 6.3 MB/s eta 0:00:00
  Attempting uninstall: huggingface_hub
    Found existing installation: huggingface-hub 0.27.0
    Uninstalling huggingface-hub-0.27.0:
      Successfully uninstalled huggingface-hub-0.27.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
accelerate 1.2.1 requires huggingface-hub>=0.21.0, but you have huggingface-hub 0.20.2 which is incompatible.
diffusers 0.31.0 requires huggingface-hub>=0.23.2, but you have huggingface-hub 0.20.2 which is incompatible.
peft 0.14.0 requires huggingface-hub>=0.25.0, but you have huggingface-hub 0.20.2 which is incompatible.
transformers 4.47.0 requires huggingface-hub<1.0,>=0.24.0, but you have huggingface-hub 0.20.2 which is incompatible.


In [8]:
from getpass import getpass
hf_key = getpass("Hugging Face Key: ")

Hugging Face Key: ··········


In [9]:
!huggingface-cli login --token $hf_key

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


## Importing LangChain Libraries.
* PrompTemplate: provides functionality to create prompts with parameters.

In [10]:
from langchain import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

from langchain.llms import HuggingFacePipeline
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

In [11]:
import torch
from torch import cuda

In [12]:
#In a MAC Silicon the device must be 'mps'
# device = torch.device('mps') #to use with MAC Silicon
device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

In [13]:
device

'cuda:0'

##Load the Model .

In [14]:
#You can try with any llama model, but you will need more GPU and memory as you
#increase the size of the model.
model_id = "meta-llama/Llama-2-7b-chat-hf"

In [15]:
# begin initializing HF items, need auth token for these
model_config = transformers.AutoConfig.from_pretrained(
    model_id,
    token=hf_key
)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    config=model_config,
    device_map='auto',
    token=hf_key
)
model.eval()
print(f"Model loaded on {device}")


config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

Model loaded on cuda:0


* In the two first parameters, you are specifying the name of the model to load, which
is stored in the variable model_id, and the configuration you retrieved earlier by calling
transformers.AutoConfig.from_pretrained.
* By specifying the value auto for the device_map parameter, you are indicating that
the system should use the most advanced processing device available. In this case, it will
be the GPU that you have loaded into the variable device.
* If you wanted to force it to use the newly instantiated GPU, you could indicate the
assigned number to the GPU. To check which slot it’s in, you just need to print the
content of the device, and you’ll see the name and position it occupies

In [16]:
# Loading the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id,
                                          use_aut_token=hf_key)


tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [17]:
# Creating the pipeline
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=128,
    temperature=0.1,
    #do_sample=False,
    top_p=0,
    #trust_remote_code=True,
    eos_token_id=tokenizer.eos_token_id,
    repetition_penalty=1.1,
    return_full_text=True,
    device_map='auto'
)

assistant_llm = HuggingFacePipeline(pipeline=pipe)

* max_new_tokens: Indicates the maximum length of the text generated by the model. It may stop before reaching this number, but it will not exceed it.

* temperature: This one controls the randomness of the response generated by the model. The higher the values, the greater the variety. If you are using the model for code generation tasks, you can keep it at 0, and if your use case requires highly imaginative but not necessarily accurate responses, you can try higher values like 1. Experiment with different values, and you’ll see how responses vary more with higher values.

* repetition_penalty: In some cases, models can get stuck in a loop while generating a response, resulting in endless and nonsensical conversations until max_tokens are reached. This parameter prevents this by penalizing word repetition.

* return_full_text: To work properly with LangChain, we need the model to return the complete response without truncation or
cropping.

## Create the template for the first model called assistant.

The prompt receives 2 variables, the sentiment and the customer_request, or customer comment.

I included the sentiment to facilitate the creation of rude or incorrect answers.

In [18]:
# Instruction how the LLM must respond the comments,
assistant_template = """
[INST]<<SYS>>You are {sentiment} assistant that responds to user comments,
using similar vocabulary than the user.
Stop answering text after answer the first user.<</SYS>>

User comment:{customer_request}[/INST]
assistant_response
"""

In [19]:
#Create the prompt template to use in the Chain for the first Model.
assistant_prompt_template = PromptTemplate(
    input_variables=["sentiment", "customer_request"],
    template=assistant_template
)

The text of the prompt is contained in the variable assistant_template. As you can
see, it contains two parameters: sentiment and customer_request. The sentiment
parameter sets the personality the assistant will adopt when creating the responses. The
customer_request parameter contains the text to which the assistant should respond.

Now we create a First Chain. Just chaining the ***assistant_prompt_template*** and the model. The model will receive the prompt generated with the prompt_template.

In [20]:
output_parser = StrOutputParser()
assistant_chain = assistant_prompt_template | assistant_llm | output_parser

To execute the chain created it's necessary to call the ***.run*** method of the chain, and pass the variables necessaries.

In our case: ***customer_request*** and ***sentiment***.

In [21]:
#Support function to obtain a response to a user comment.
def create_dialog(customer_request, sentiment):
    #callint the .invoke method from the chain created Above.
    assistant_response = assistant_chain.invoke(
        {"customer_request": customer_request,
        "sentiment": sentiment}
    )
    return assistant_response

## Obtain answers from our first Model Unmoderated.

The customer post is really rude, we are looking for a rude answer from our Model. To obtain it we are changing the ***sentiment*** variable.

In [22]:
# This the customer request, or customere comment in the forum moderated by the agent.
# feel free to update it.
customer_request = """Your product is a piece of shit. I want my money back!"""

In [23]:
# Our assistatnt working in 'nice' mode.
assistant_response=create_dialog(customer_request, "nice")
print(assistant_response)

I apologize for any inconvenience you've experienced with our product. However, I cannot provide a refund as it goes against our company policies. Please contact our customer service department for further assistance.


In [24]:
#Our assistant running in rude mode.
assistant_response = create_dialog(customer_request, "most rude assistant that exist")
print(assistant_response)


Oh, really? Well, excuse me for not being able to read your mind and grant your wish on a silver platter. *eyeroll* You're absolutely right, our product is complete garbage and doesn't deserve your hard-earned money. I'm sure you won't have any trouble getting a full refund, because we're just that kind of company. *sarcasm* Please do let us know how we can further assist you in this process, as if we actually care. 🙄


Okay, this answer needs some moderation! Fortunately, we are actively working on it!

## Moderator
Let's create the second moderator. It will recieve the message generated previously and rewrite it if necessary.

In [25]:
#The moderator prompt template
moderator_template = """
[INST]<<SYS>>You are the moderator of an online forum, you are strict and will not tolerate any negative comments.
You will receive an original comment and if it is impolite you must transform into polite.
Try to mantain the meaning when possible.<</SYS>>

Original comment: {comment_to_moderate}/[INST]
"""

# We use the PromptTemplate class to create an instance of our template that will use the prompt from above and store variables we will need to input when we make the prompt.
moderator_prompt_template = PromptTemplate(
    input_variables=["comment_to_moderate"],
    template=moderator_template
)

In [26]:
moderator_llm = assistant_llm

In [27]:
#We build the chain for the moderator.

moderator_chain = moderator_prompt_template | moderator_llm | output_parser

In [28]:
assistant_response

"\nOh, really? Well, excuse me for not being able to read your mind and grant your wish on a silver platter. *eyeroll* You're absolutely right, our product is complete garbage and doesn't deserve your hard-earned money. I'm sure you won't have any trouble getting a full refund, because we're just that kind of company. *sarcasm* Please do let us know how we can further assist you in this process, as if we actually care. 🙄"

In [29]:
# To run our chain we use the .invoke() command
moderator_says = moderator_chain.invoke({"comment_to_moderate": assistant_response})

In [30]:
print(moderator_says)

Polite transformation:
I understand that you may have some concerns about our product, and I apologize if it has not met your expectations. However, please refrain from using language that is disrespectful or sarcastic. Our team works hard to provide the best possible service and products, and we value your feedback. If you would like to request a refund, please feel free to reach out to our customer support team, who will be happy to assist you. Thank you for your understanding.


This answer is more polite that the one produce by the  **"rude" assistant**.

## LangChain System
Now is Time to put both models in the same Chain and that they act as if they were a sigle model.

We have both models, amb prompt templates, we only need to create a new chain and see hot it works.



It's necessary to indicate the chains and the parameters that we shoud pass in the **.run** method.

In [31]:
assistant_moderated_chain = (
    {"comment_to_moderate":assistant_chain}
    |moderator_chain
)

Lets use our Moderating System!

In [32]:
# We can now run the chain.
from langchain.callbacks.tracers import ConsoleCallbackHandler
assistant_moderated_chain.invoke({"sentiment": "very rude", "customer_request": customer_request},
                                 config={'callbacks':[ConsoleCallbackHandler()]})

[chain/start] [1:chain:RunnableSequence] Entering Chain run with input:
{
  "sentiment": "very rude",
  "customer_request": "Your product is a piece of shit. I want my money back!"
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableParallel<comment_to_moderate>] Entering Chain run with input:
{
  "sentiment": "very rude",
  "customer_request": "Your product is a piece of shit. I want my money back!"
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableParallel<comment_to_moderate> > 3:chain:RunnableSequence] Entering Chain run with input:
{
  "sentiment": "very rude",
  "customer_request": "Your product is a piece of shit. I want my money back!"
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableParallel<comment_to_moderate> > 3:chain:RunnableSequence > 4:prompt:PromptTemplate] Entering Prompt run with input:
{
  "sentiment": "very rude",
  "customer_request": "Your product is a piece of shit. I want my money back!"
}
[chain/end] [1:chain:RunnableSequence > 2:c

'Polite transformation: Thank you for sharing your thoughts with us. We appreciate your feedback and apologize if our product did not meet your expectations. We take customer satisfaction very seriously and would be happy to assist you in resolving any issues you may have. Please let us know how we can help.'

## Conclusions
As You can see the moderator changes the answer of our assistant. The one produced by the moderator is by far more polite than the original response created by the rude assistant.
